### Necessary imports

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import cv2
from IPython.display import Video
from torchvision.io import read_video, read_video_timestamps
import csv
import av
import json

import tensorflow as tf

### Append video dataset paths and JSON path

In [2]:
train_dir = os.path.normpath(os.getcwd() + "/../Video processing" + os.sep + "video_small" + os.sep + "train")
val_dir = os.path.normpath(os.getcwd() +  "/../Video processing" + os.sep + "video_small" + os.sep + "val")
sys.path.append(train_dir)
sys.path.append(val_dir)

In [3]:
path_to_data = os.path.normpath(os.getcwd() + "/../Video processing" + os.sep + "video_small/")

In [4]:
info_json_path = os.path.join(os.getcwd() + "/../Video processing" + os.sep + "video_small" + os.sep + "smaller_transition_times.json")
info_json = json.load(open(info_json_path))

In [5]:
test_video = "train/oops/25 Best Trampoline Fail Nominees - FailArmy Hall of Fame (July 2017)23.mp4"
test_key = "25 Best Trampoline Fail Nominees - FailArmy Hall of Fame (July 2017)23"
video_path = os.path.join(path_to_data, test_video)
Video(video_path, embed=True, width=600)

In [7]:
test_video = "train/no_oops/Armpit Wax - Fails You Missed (April 2018) _ 201825.mp4"
test_key = "Armpit Wax - Fails You Missed (April 2018) _ 201825"
video_path = os.path.join(path_to_data, test_video)
Video(video_path, embed=True, width=600)

In [8]:
def get_label_time(key, info_json):
  times = info_json[key]['t']
  if min(times) < 0:
    return -1
  else:
    return np.mean(times)
# from DLM - batch (dict): batch of data batch['obs'].shape = (B, T, Obs_dim)
#                     and batch['actions'].shape = (B, T, Action_dim)

def trim_video(video_tensor, video_timestamps, time_limit):
  video_tensor = np.array(video_tensor)
  video_timestamps = np.array(video_timestamps)
  video_tensor = video_tensor[video_timestamps < time_limit]
  video_timestamps = video_timestamps[video_timestamps < time_limit]
  return video_tensor, video_timestamps

def subsample_video(video_tensor, video_timestamps, video_fps, hz, num_frames=25):
  fps_ratio = round(video_fps / hz)
  frame_samples = [round(x * fps_ratio) for x in range(num_frames)]
  video_tensor = np.array(video_tensor)
  video_timestamps = np.array(video_timestamps)
  if video_tensor.shape[0] <= frame_samples[-1]:
    frame_samples = frame_samples[:-1]
  # print(frame_samples)
  # video_tensor = video_tensor[::fps_ratio]
  new_video_tensor = video_tensor[frame_samples]
  # video_timestamps = video_timestamps[::fps_ratio]
  new_video_timestamps = video_timestamps[frame_samples]
  if new_video_tensor.shape[0] > num_frames:
    new_video_tensor = new_video_tensor[:num_frames]
    new_video_timestamps = new_video_timestamps[:num_frames]
  elif new_video_tensor.shape[0] == num_frames-1:
    new_video_tensor = np.append(new_video_tensor, np.array([video_tensor[-1]]), axis=0)
    new_video_timestamps = np.append(new_video_timestamps, np.array([video_timestamps[-1]]), axis=0)
  else:
    print(new_video_tensor.shape[0])
  return new_video_tensor, new_video_timestamps

def match_labels_to_frames(video_tensor, video_timestamps, time):
  if time < 0:
    time = 100000000000000
  # else:
  #   time_ms = time
  labels = np.where(np.array(video_timestamps) < time, 0, 1)
  batch = {'frames': video_tensor, 'labels': labels}
  return batch

def grayscale_video(video_tensor):
  video_tensor = np.array(video_tensor)
  video_tensor = np.array(tf.image.rgb_to_grayscale(video_tensor))
  # video_tensor = video_tensor.squeeze()
  return video_tensor

def reshape_video(video_tensor, size):
  video_tensor = np.array(video_tensor)
  n, d1, d2, c = video_tensor.shape
  if d1 < d2:
    y = d1
    x = d2
    v_offset = 0
    h_offset = int((x/2) - (y/2))
  else:
    y = d2
    x = d1
    v_offset = int((x/2) - (y/2))
    h_offset = 0
  video_tensor = tf.image.crop_to_bounding_box(video_tensor, v_offset, h_offset, y, y)
  video_tensor = tf.image.resize(video_tensor, size)
  video_tensor = np.array(video_tensor)
  return video_tensor

In [9]:
def save_video(video_dict, path, video_name):
  if not os.path.exists(path):
    os.makedirs(path)
  video_tensor = video_dict['frames']
  labels = video_dict['labels']
  # Save Images
  for i in range(video_tensor.shape[0]):
    image = video_tensor[i]
    save_path = path + '/' + video_name + '_' + str(i).zfill(2) + '.jpg'
    # print(save_path)
    cv2.imwrite(save_path, image)
  # Save Labels
  labels_path = path + '/' + video_name + '_labels.csv'
  with open(labels_path, 'a', newline='') as csvfile:
    label_writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    label_writer.writerow(labels)

In [10]:
def process_video(video_name, video_path, info_json, time_limit=5.5, shape=(240, 240), hz=5):
  # Read video from path and get tensor and timestamps
  v_tensor, _, fps_info = read_video(video_path, pts_unit='sec')
  v_timestamps, video_fps = read_video_timestamps(video_path, pts_unit='sec')
  # Get label change time
  label_time = get_label_time(video_name, info_json)
  # Process video
  v_tensor, v_timestamps = subsample_video(v_tensor, v_timestamps, video_fps, hz, num_frames=25) # Subsample
  # v_tensor, v_timestamps = trim_video(v_tensor, v_timestamps, time_limit) # Trim
  if v_tensor.shape[0] != 25:
    raise Exception("Video " + video_name + " is not 25 frames long! It is " + str(v_tensor.shape[0]) + " frames long.")
  v_tensor = grayscale_video(v_tensor) # Grayscale
  v_tensor = reshape_video(v_tensor, shape) # Reshape
  # print(v_tensor.shape)
  frames_and_labels = match_labels_to_frames(v_tensor, v_timestamps, label_time) # Get labels
  return frames_and_labels

In [11]:
def process_dataset(data_path, new_data_path, video_json):
  count = 0
  for video in video_json.keys():
    # Get path for video in json
    category = video_json[video]['category']
    label = video_json[video]['label']
    video_path = os.path.join(data_path, category, label, video) + '.mp4'
    save_path = os.path.join(new_data_path, category)
    if os.path.isfile(save_path + '/' + video + '_labels.csv'):
      count += 1
      print("Video", count, "already exists. Skipping.")
      continue
    # if label == 'no_oops':
    #   continue
    # Convert video into tensor
    frames_and_labels = process_video(video, video_path, video_json, time_limit=5, shape=(240, 240))
    # Save tensor to processed images and label csv in new data path
    save_video(frames_and_labels, save_path, video)
    count += 1
    print("Saved video", count, "out of", len(video_json.keys()), "-", video)

In [12]:
data_path = path_to_data
new_data_path = os.path.normpath(os.getcwd() + os.sep + "video_proc/")
# new_data_path = '/content/temp/video_proc_ii'
json_file = 'smaller_transition_times.json'
video_json = info_json

In [13]:
process_dataset(data_path, new_data_path, video_json)

25
Saved video 1 out of 1255 - 25 Best Trampoline Fail Nominees - FailArmy Hall of Fame (July 2017)18
25
Saved video 2 out of 1255 - 28 Best Skateboard Fail Nominees - FailArmy Hall of Fame (August 2017)18
25
Saved video 3 out of 1255 - 29 Best Work Fail Nominees - FailArmy Hall of Fame (August 2017)1
25
Saved video 4 out of 1255 - 34 Workout Fail Nominees - FailArmy Hall Of Fame (April 2017)8
25
Saved video 5 out of 1255 - A Boatload of Fails - Throwback Fails (November 2017) _ FailArmy12
25
Saved video 6 out of 1255 - A Boatload of Fails - Throwback Fails (November 2017) _ FailArmy95
25
Saved video 7 out of 1255 - American Fails (July 2017) _ FailArmy37
25
Saved video 8 out of 1255 - Animals Can Be Jerks! (June 2018) _ FailArmy6
25
Saved video 9 out of 1255 - Armpit Wax - Fails You Missed (April 2018) _ 201825
25
Saved video 10 out of 1255 - Backflipping Bros - Fails of the Month! (May 2018) _ Failarmy14
25
Saved video 11 out of 1255 - Backflipping Bros - Fails of the Month! (May 201